In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [6]:
books = pd.read_csv('books_with_categories.csv')

In [7]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [8]:
classifier(books['description'][0])

[[{'label': 'anger', 'score': 0.003935146611183882},
  {'label': 'disgust', 'score': 0.019100705161690712},
  {'label': 'fear', 'score': 0.6548413634300232},
  {'label': 'joy', 'score': 0.015161297284066677},
  {'label': 'neutral', 'score': 0.16985207796096802},
  {'label': 'sadness', 'score': 0.11640888452529907},
  {'label': 'surprise', 'score': 0.02070062793791294}]]

In [9]:
classifier(books['description'][0].split('.'))

[[{'label': 'anger', 'score': 0.009156348183751106},
  {'label': 'disgust', 'score': 0.00262847519479692},
  {'label': 'fear', 'score': 0.06816219538450241},
  {'label': 'joy', 'score': 0.04794241115450859},
  {'label': 'neutral', 'score': 0.1403856873512268},
  {'label': 'sadness', 'score': 0.0021221605129539967},
  {'label': 'surprise', 'score': 0.7296026945114136}],
 [{'label': 'anger', 'score': 0.040478333830833435},
  {'label': 'disgust', 'score': 0.2735914885997772},
  {'label': 'fear', 'score': 0.006879060063511133},
  {'label': 'joy', 'score': 0.10908304899930954},
  {'label': 'neutral', 'score': 0.44937071204185486},
  {'label': 'sadness', 'score': 0.09362724423408508},
  {'label': 'surprise', 'score': 0.02697017975151539}],
 [{'label': 'anger', 'score': 0.01103188470005989},
  {'label': 'disgust', 'score': 0.04342261329293251},
  {'label': 'fear', 'score': 0.014084079302847385},
  {'label': 'joy', 'score': 0.014211442321538925},
  {'label': 'neutral', 'score': 0.6462162137031

In [10]:
emotion_labels=['anger','disgust','fear','joy','neutral','sadness','surprise']
isbn =[]
emotion_scores={label:[] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
  per_emotion_scores={label:[] for label in emotion_labels}
  for prediction in predictions:
    sorted_prediction = sorted(prediction,key=lambda x:x['label'])
    for index,label in enumerate(emotion_labels):
      per_emotion_scores[label].append(sorted_prediction[index]['score'])
  return {label:np.max(scores) for label,scores in per_emotion_scores.items()}


In [11]:
for i in range(len(books)):
  isbn.append(books['isbn13'][i])
  sentences = books['description'][i].split('.')
  predictions= classifier(sentences)
  max_scores = calculate_max_emotion_scores(predictions)
  for label in emotion_labels :
    emotion_scores[label].append(max_scores[label])




You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [12]:
emotion_scores

{'anger': [0.0641336739063263,
  0.6126185059547424,
  0.0641336739063263,
  0.35148391127586365,
  0.08141247183084488,
  0.2322249710559845,
  0.5381842255592346,
  0.0641336739063263,
  0.30067017674446106,
  0.0641336739063263,
  0.0641336739063263,
  0.6115744113922119,
  0.10621520131826401,
  0.0641336739063263,
  0.07239244878292084,
  0.09586354345083237,
  0.16215930879116058,
  0.0641336739063263,
  0.08193845301866531,
  0.2234596312046051,
  0.1071522980928421,
  0.0641336739063263,
  0.0641336739063263,
  0.0641336739063263,
  0.3963673412799835,
  0.06472665071487427,
  0.0641336739063263,
  0.0641336739063263,
  0.0641336739063263,
  0.30293354392051697,
  0.06683351844549179,
  0.0641336739063263,
  0.0641336739063263,
  0.16889671981334686,
  0.0641336739063263,
  0.0641336739063263,
  0.0641336739063263,
  0.0791032686829567,
  0.11856507509946823,
  0.0641336739063263,
  0.5636678338050842,
  0.9171825051307678,
  0.06835673004388809,
  0.2205992043018341,
  0.06413

In [15]:
emotion_df = pd.DataFrame(emotion_scores)
emotion_df['isbn13'] = isbn

In [16]:
books = pd.merge(books,emotion_df,on='isbn13')


In [17]:
books.to_csv('books_with_emotion.csv',index=False)